Week1 Homework: 
Introduction and Elastic Search for practice

Q1. Running Elastic
- Run Elastic Search 8.4.3,
- curl localhost:9200
- version.build_hash value is : "42f05b9372a9a4a470db3b52817899b99a76ee73"


In [2]:
#Getting the data
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

Q2. Indexing the data
- index() function is used to add the data to elastic search

In [4]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "llm-hw1"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'llm-hw1'})

In [6]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

Q3. Searching
- The score of top ranking result is: 84.05

In [11]:
user_question = 'How do I execute a command in a running docker container?'

In [12]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)
response

ObjectApiResponse({'took': 469, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'llm-hw1', '_id': 't4kkRpABz862w5AKwCjv', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'llm-hw1', '_id': 'xYkkRpABz862w5AKDiYv', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead

Q4. Filtering

The 3rd question returned by the search engine is:

-How do I copy files from a different folder into docker container’s working directory?


In [13]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text","section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)
documents = [hit['_source'] for hit in response['hits']['hits']]
documents


[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

Q5. Building a prompt
-the length of the prompt is 1462

In [14]:
#Building context
context_template= ""

for doc in documents:
    doc_str = f"Question: {doc['question']}\nAnswer: {doc['text']}\n\n"
    context_template+= doc_str

context_template = context_template.strip()
print(context_template)

Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Question: How do I copy files from a different folder into docker container’s working directory?
Answer: You can copy files from your local machine into a Docker container using t

In [31]:
#Building actual prompt to prevent hallucinations
prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {user_question}

CONTEXT:
{context_template}
""".strip()

In [26]:
print(len(prompt))

1462


Q6. Tokens in OpenAI
- 322 tokens

In [32]:
import os
import openai
import tiktoken


In [33]:
encoding = tiktoken.encoding_for_model("gpt-4o")
prompt_to_token = encoding.encode(prompt)

In [34]:
number_of_tokens = len(prompt_to_token)
print(number_of_tokens)

322


Bonus: generating the answer (ungraded)
Let's send the prompt to OpenAI. What's the response?


In [36]:
from openai import OpenAI

In [37]:
#api_key is not exposed
client = OpenAI(api_key)

In [38]:

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [39]:
answer = llm(prompt)

In [40]:
print(answer)

To execute a command in a running Docker container, you can use the `docker exec` command. Here is the step-by-step process:

1. First, find the `container-id` of the running container by using the `docker ps` command:
   ```sh
   docker ps
   ```

2. Once you have the `container-id`, you can execute a command in the container. For example, if you want to start a bash session inside the container, use:
   ```sh
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of your running container.


Bonus: calculating the costs (ungraded)
Suppose that on average per request we send 150 tokens and receive back 250 tokens.
On June 17, the prices for gpt4o are:

Input: $0.005 / 1K tokens
Output: $0.015 / 1K tokens

How much will it cost to run 1000 requests?
- (150 x 0.005) + (250 x 0.015) = $4.5



In [41]:
(150 * 0.005) + (250 * 0.015) 

4.5

For homework : input 322 tokens output: 130 token 
-  (0.005 +  0.015) = $0.02 

In [43]:
answer_to_token = encoding.encode(answer)
number_of_answer_tokens = len(answer_to_token)
print(number_of_tokens)
print(number_of_answer_tokens)


322
130
